In [1]:
import numpy as np
import praw
from nltk import tokenize
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import pandas as pd

# bokeh
from bokeh.io import push_notebook, show, output_notebook
from bokeh.models import ColumnDataSource, HoverTool, PanTool
from bokeh.plotting import figure, show

from CryptoSentWebApp import config

In [3]:
conf = config.get_secret('tst/CryptoSentWebApp/Reddit')

In [5]:
output_notebook()

Loading BokehJS ...

In [6]:
sid = SentimentIntensityAnalyzer()

In [7]:
reddit = praw.Reddit(
    client_id = conf['client_id'],
    client_secret = conf['client_secret'],
    username = conf['username'],
    password = conf['password'],
    user_agent = conf['user_agent']
)

In [17]:
subreddit = reddit.subreddit('cryptocurrency')
for comment in subreddit.stream.comments():
    print(comment.parent().body)
    comment_sent = tokenize.sent_tokenize(comment.body)
    comment_ts = pd.to_datetime(comment.created_utc, unit="s")
    for sentence in comment_sent:
        # Extract the subject(s) from the sentence

        # Extract the sentiment from the sentence
        sent = sid.polarity_scores(sentence)
#         sent_ma = moving_average(data.data['sentiment'], 50)
        print(dict(
            timestamp=[comment_ts],
            sentence=[sentence],
            sentiment=[sent['compound']],
#             sentiment_ma=[sent_ma]
        ))

did you want to fast forward to your point ??
{'timestamp': [Timestamp('2020-03-21 06:07:58')], 'sentence': ["The point was that Ethereum's share of token market capitalization has reached near absolute dominance."], 'sentiment': [0.5267]}
{'timestamp': [Timestamp('2020-03-21 06:07:58')], 'sentence': ["That's noteworthy, and speaks to Ethereum's increasingly overwhelming and insurmountable network effect advantage over all other platforms."], 'sentiment': [0.25]}
{'timestamp': [Timestamp('2020-03-21 06:07:58')], 'sentence': ['To put it more succinctly, Ethereum is emerging as the internet of value.'], 'sentiment': [0.34]}
Why has ETH been so stagnant?
{'timestamp': [Timestamp('2020-03-21 06:09:16')], 'sentence': ['Bluntly put, Its now clear to everybody nobody wants a turing complete blockchain, except for die hard "fans".'], 'sentiment': [-0.4019]}


AttributeError: 'Submission' object has no attribute 'body'

In [8]:
def moving_average(window, t=10, start=0):
    if len(window) < t: 
        return [start]
    return np.convolve(window[-t:], np.ones(t, dtype=float), mode="valid") / t

In [9]:
def subreddit_sentiment(subreddit):
    data = ColumnDataSource(dict(timestamp=[], sentence=[], sentiment=[], sentiment_ma=[]))    
    hover = HoverTool(
        tooltips=[('time', '@timestamp{%Y-%m-%d %H:%M:%S}'), ('sentiment', '@sentiment_ma{%0.3f}')],
        formatters={'timestamp': 'datetime', 'sentiment_ma': 'printf'},
        mode='vline'
    )
    p = figure(plot_width=800,
               plot_height=400,
               x_axis_type='datetime',
               tools=[hover, PanTool(dimensions='height')],
               title="Real-Time sentiment plot")

    r = p.line(source=data, x='timestamp', y='sentiment_ma')
    p.xaxis.axis_label = "Time"
    p.yaxis.axis_label = "Reddit comment sentiment score"

    target = show(p, notebook_handle=True)
    for comment in subreddit.stream.comments():
    #     print(comment.parent().body)
        comment_sent = tokenize.sent_tokenize(comment.body)
        comment_ts = pd.to_datetime(comment.created_utc, unit="s")
        for sentence in comment_sent:
            # Extract the subject(s) from the sentence

            # Extract the sentiment from the sentence
            sent = sid.polarity_scores(sentence)
            sent_ma = moving_average(data.data['sentiment'], 50)
            data.stream(dict(timestamp=[comment_ts],
                             sentence=[sentence],
                             sentiment=[sent['compound']],
                             sentiment_ma=[sent_ma]), 
                        1000)
            push_notebook(handle=target)

In [10]:
subreddit_sentiment('cryptocurrency')

KeyboardInterrupt: 

In [11]:
sid = SentimentIntensityAnalyzer()
for submission in subreddit.stream.submissions():
    sent = sid.polarity_scores(submission.title)
    print(submission.title, sent)

NameError: name 'subreddit' is not defined